In [1]:
import tensorflow as tf


In [2]:
dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"


In [3]:
dataset_url

'https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz'

In [9]:
zip_data=tf.keras.utils.get_file(
    origin=dataset_url,
    fname='flowers_photos.tgz',
    extract=True
)
zip_data

228813984/228813984 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


'/root/.keras/datasets/flowers_photos_extracted'

In [10]:
import os

In [35]:
import tensorflow as tf
import os

dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"

zip_data = tf.keras.utils.get_file(
    origin=dataset_url,
    fname='flower_photos.tgz',
    extract=True
)

# Correctly extract the base directory
# os.path.dirname(zip_data) gives the directory containing the zip file
# os.path.splitext(os.path.basename(zip_data))[0] gives the filename without extension
base_directory = os.path.join(os.path.dirname(zip_data), os.path.splitext(os.path.basename(zip_data))[0])

data_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1.0 / 255.0,
    validation_split=0.2
)

batchsize = 64

training_data = data_generator.flow_from_directory(
    base_directory,
    target_size=(224, 224),
    batch_size=batchsize,
    subset='training'
)

Found 2936 images belonging to 1 classes.


In [40]:
testing_data = data_generator.flow_from_directory(
    base_directory,
    target_size=(224, 224),
    batch_size=batchsize,
    subset='validation'
)

Found 734 images belonging to 1 classes.


In [41]:
training_data.class_indices

{'flower_photos': 0}

In [42]:
model = tf.keras.models.Sequential(
    [
     tf.keras.layers.Conv2D(
         64,
         (3, 3),
         activation = 'relu',
         input_shape = (224, 224, 3)
     ),
     tf.keras.layers.MaxPooling2D(2, 2),
     tf.keras.layers.Conv2D(
         64,
         (3, 3),
         activation = 'relu'
     ),
     tf.keras.layers.MaxPooling2D(2, 2),
     tf.keras.layers.Conv2D(
         64,
         (3, 3),
         activation = 'relu'
     ),
     tf.keras.layers.MaxPooling2D(2, 2),
     tf.keras.layers.Flatten(),
     tf.keras.layers.Dense(
         512,
         activation = 'relu'
     ),
     tf.keras.layers.Dense(
         5,
         activation = 'softmax'
     )
    ]
)


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [43]:

model.compile(
    loss = 'categorical_crossentropy',
    optimizer = 'sgd',
    metrics = ['accuracy']
)

In [44]:
training_steps = training_data.samples // batchsize
testing_steps = testing_data.samples // batchsize

In [46]:
model.fit(
    training_data,
    validation_data = testing_data,
    epochs = 10,
    steps_per_epoch = training_steps,
    validation_steps = testing_steps,
)

Epoch 1/10


ValueError: Exception encountered when calling Sequential.call().

[1mInput 0 of layer "dense" is incompatible with the layer: expected axis -1 of input shape to have value 43264, but received input with shape (None, 25600)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(None, 180, 180, 3), dtype=float32)
  • training=True
  • mask=None

In [47]:
import tensorflow as tf

model = tf.keras.models.Sequential(
    [
     tf.keras.layers.Conv2D(
         64,
         (3, 3),
         activation = 'relu',
         input_shape = (224, 224, 3)
     ),
     tf.keras.layers.MaxPooling2D(2, 2),
     tf.keras.layers.Conv2D(
         64,
         (3, 3),
         activation = 'relu'
     ),
     tf.keras.layers.MaxPooling2D(2, 2),
     tf.keras.layers.Conv2D(
         64,
         (3, 3),
         activation = 'relu'
     ),
     tf.keras.layers.MaxPooling2D(2, 2),
     tf.keras.layers.Flatten(),
     # The input shape for the Dense layer was incorrect.
     # It should be calculated based on the output of the previous layers.
     # Since we don't know the exact output shape, we can use a GlobalAveragePooling2D layer
     # to reduce the spatial dimensions to 1x1 and then feed it to the Dense layer.
     tf.keras.layers.GlobalAveragePooling2D(), # Added this layer
     tf.keras.layers.Dense(
         512,
         activation = 'relu'
     ),
     tf.keras.layers.Dense(
         5,
         activation = 'softmax'
     )
    ]
)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


ValueError: Input 0 of layer "global_average_pooling2d" is incompatible with the layer: expected ndim=4, found ndim=2. Full shape received: (None, 43264)